DATA PREPROCESSING

In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

Load data

In [3]:
train_data = pd.read_csv("Dataset/train.csv", delimiter=",")
test_data = pd.read_csv("Dataset/test.csv", delimiter=",")
data = pd.concat([train_data, test_data])
data.reset_index(inplace=True)
data[:]
data = data.dropna(subset=['Fare', 'Embarked'])
data.describe()


,index,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,1306.000000,1306.000000,889.000000,1306.000000,1043.000000,1306.00000,1306.000000,1306.000000
mean,369.529862,655.022205,0.382452,2.296325,29.813202,0.50000,0.385911,33.223956
std,248.508278,377.913727,0.486260,0.837037,14.366254,1.04258,0.866357,51.765986
min,0.000000,1.000000,0.000000,1.000000,0.170000,0.00000,0.000000,0.000000
25%,164.000000,328.250000,0.000000,2.000000,21.000000,0.00000,0.000000,7.895800
50%,327.000000,654.500000,0.000000,3.000000,28.000000,0.00000,0.000000,14.454200
75%,562.750000,981.750000,1.000000,3.000000,39.000000,1.00000,0.000000,31.275000
max,890.000000,1309.000000,1.000000,3.000000,80.000000,8.00000,9.000000,512.329200


Data Preprocessing 

1. Name feature

In [4]:
titles = set()
for name in data['Name']:
    titles.add(name.split(',')[1].split('.')[0].strip())

Title_Dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Royalty"}

# Edatatract the title from each name
data['Title'] = data['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())
data['Title'] = data.Title.map(Title_Dictionary)

In [5]:
le_title = preprocessing.LabelEncoder()
le_title.fit(['Officer', 'Royalty', 'Mrs', 'Mr', 'Master', 'Miss'])
data['Title'] = le_title.transform(data['Title'])

2. Age feature

In [6]:
grouped_train = data.iloc[:891].groupby(['Sex', 'Pclass', 'Title'])
# Median
grouped_train_median = grouped_train.median('Age')
grouped_train_median = grouped_train_median.reset_index()[['Sex', 'Pclass', 'Title', 'Age']]

def fill_age(row):
    condition = (
        (grouped_train_median['Sex'] == row['Sex']) & 
        (grouped_train_median['Title'] == row['Title']) & 
        (grouped_train_median['Pclass'] == row['Pclass'])
    ) 
    return grouped_train_median[condition]['Age'].values[0]

data['Age'] = data.apply(lambda row: fill_age(row) if np.isnan(row['Age']) else row['Age'], axis = 1)


3. Fare feature

4. Embraked feature

In [7]:
# Initialize and fit LabelEncoder
le_embarked = preprocessing.LabelEncoder()
le_embarked.fit(['Q', 'S', 'C'])

# Transform 'Embarked' column
data['Embarked'] = le_embarked.transform(data['Embarked'])

5. Cabin feature

In [8]:
data.fillna('U', inplace=True)
data['Cabin'] = data['Cabin'].map(lambda c: c[0])
cabin_values = data['Cabin'].unique()
le_cabin = preprocessing.LabelEncoder()
le_cabin.fit(cabin_values)
data['Cabin'] = le_cabin.transform(data['Cabin'])


6. Sex feature

In [9]:
data['Sex'] = data['Sex'].map({'male':1, 'female':0})

In [12]:
data.describe()

,index,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Title
count,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.00000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000
mean,369.529862,655.022205,2.296325,0.644717,29.159778,0.50000,0.385911,33.223956,6.737366,1.492343,1.908116
std,248.508278,377.913727,0.837037,0.478782,13.353240,1.04258,0.866357,51.765986,2.446816,0.814817,0.801805
min,0.000000,1.000000,1.000000,0.000000,0.170000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,164.000000,328.250000,2.000000,0.000000,21.000000,0.00000,0.000000,7.895800,8.000000,1.000000,2.000000
50%,327.000000,654.500000,3.000000,1.000000,26.000000,0.00000,0.000000,14.454200,8.000000,2.000000,2.000000
75%,562.750000,981.750000,3.000000,1.000000,36.000000,1.00000,0.000000,31.275000,8.000000,2.000000,2.000000
max,890.000000,1309.000000,3.000000,1.000000,80.000000,8.00000,9.000000,512.329200,8.000000,2.000000,5.000000


Save dataset

In [10]:
train_data = data.iloc[:891]
test_data = data.iloc[891:]
train_data.to_csv('Dataset/train_preprocessing.csv')
test_data.to_csv('Dataset/test_preprocessing.csv')